In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import xesmf as xe
import nctoolkit as nc

# import matplotlib.pyplot as plt
# import seaborn as sns; sns.set()
# import cartopy.crs as ccrs
# from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

nctoolkit is using Climate Data Operators version 2.0.5


In [2]:
nc.deep_clean()

# Collect datasets and regrid to a common 1x1 degree grid

In [3]:
# dataset configuration
date_range = pd.date_range(start="2014-09", end="2017-04", freq="1M")
target_grid = xr.Dataset(
    {
        "lat": (["lat"], np.arange(-89.5, 90.5, 1.0)),
        "lon": (["lon"], np.arange(-180, 180, 1.0)),
    }
)
# xr.DataArray(None, coords=target_grid).to_netcdf("../data/target_grid.nc")

# # figure out how to compute 1-degree areas
# ds_nc = nc.from_xarray(xr.DataArray(None, coords=target_grid))
# ds_nc = nc.open_data("../data/target_grid.nc")
# ds_nc.set_gridtype("lonlat")
# ds_nc.cell_area(join=False)
# grid_area = ds_nc.to_xarray()
# grid_area

In [4]:
mozart_paths = []
for month in date_range:
    yyyy, mm, _ = str(month).split("-")
    mozart_paths.append(
        f"../1_transport/intermediates/MOZART/output/BasisFnsUpdated/{yyyy}{mm}/"
        f"BasisFnsUpdated.mz4.h0.{yyyy}-{mm}-01-03600.nc"
    )

In [5]:
with xr.open_mfdataset(mozart_paths, decode_times=False) as ds:
    # setup times correctly
    ds["time"] = pd.to_datetime(ds["date"].values, format="%Y%m%d") + pd.to_timedelta(
        ds["datesec"].values, unit="seconds"
    )
    # compute pressure edge
    ds["pressure_edge"] = (
        ds["P0"] * ds["hyai"] + ds["PS"] * ds["hybi"]
    ) / 100
    ds_mozart = ds[["CO2_VMR_avrg", "pressure_edge"]]
     # shift longitude coordinate reference
    ds_mozart["lon"] = ds_mozart["lon"] - 180.0
    # regrid to 1x1 degree
    ds_mozart = xe.Regridder(ds_mozart, target_grid, "bilinear")(ds_mozart)

# one observation for every hour of every day in study period, in units VMR, regridded
ds_mozart


,Array,Chunk
Bytes,305.95 GiB,10.06 GiB
Shape,"(22632, 56, 180, 360)","(744, 56, 180, 360)"
Count,126 Tasks,31 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,311.41 GiB,10.24 GiB
Shape,"(22632, 57, 180, 360)","(744, 57, 180, 360)"
Count,778 Tasks,31 Chunks
Type,float32,numpy.ndarray


In [6]:
mozart_datetime_end = ds_mozart.time.values.max()

geoschem_level_edge_glob = (
    "../1_transport/intermediates/GEOS_Chem/runs/run.v12.3.2.base/output/"
    "GEOSChem.LevelEdgeDiags.*_0000z.nc4"
)
with xr.open_mfdataset(geoschem_level_edge_glob) as ds:
    da_pressure_level = ds["Met_PEDGE"]

geoschem_spec_conc_glob = (
    "../1_transport/intermediates/GEOS_Chem/runs/run.v12.3.2.base/output/"
    "GEOSChem.SpeciesConc.*_0000z.nc4"
)
with xr.open_mfdataset(geoschem_spec_conc_glob) as ds:
    ds_geoschem = ds[["SpeciesConc_CO2", "AREA"]]
    ds_geoschem["Met_PEDGE"] = da_pressure_level
    # clip to available mozart date range
    ds_geoschem = ds_geoschem.where(ds_geoschem["time"] <= mozart_datetime_end, drop=True)
    # regrid to 1x1 degree
    ds_geoschem = xe.Regridder(ds_geoschem, target_grid, "bilinear")(ds_geoschem)

# one observation for every hour of every day in study period, in units mole per mole dry, regridded
ds_geoschem

<xarray.Dataset>
Dimensions:          (time: 22632, lev: 47, lat: 180, lon: 360, ilev: 48)
Coordinates:
  * time             (time) datetime64[ns] 2014-09-01T00:30:00 ... 2017-03-31...
  * lev              (lev) float64 0.9925 0.9775 0.9625 ... 0.0001387 3.8e-05
  * ilev             (ilev) float64 1.0 0.985 0.97 ... 0.0002113 6.6e-05 1e-05
  * lon              (lon) float64 -180.0 -179.0 -178.0 ... 177.0 178.0 179.0
  * lat              (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
Data variables:
    SpeciesConc_CO2  (time, lev, lat, lon) float32 dask.array<chunksize=(24, 47, 180, 360), meta=np.ndarray>
    AREA             (time, lat, lon) float32 dask.array<chunksize=(24, 180, 360), meta=np.ndarray>
    Met_PEDGE        (time, ilev, lat, lon) float32 dask.array<chunksize=(24, 48, 180, 360), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

## Compute vertical averages for the first timestep and map the difference

In [14]:
ds_mozart["pressure_edge"]["ilev"].values

array([   1.65079  ,    2.08497  ,    2.6202111,    3.276431 ,
          4.076571 ,    5.046801 ,    6.2168007,    7.619842 ,
          9.292941 ,   11.2769   ,   13.6434   ,   16.4571   ,
         19.7916   ,   23.73041  ,   28.36781  ,   33.81001  ,
         40.17541  ,   47.64391  ,   56.38791  ,   66.60341  ,
         78.512314 ,   92.36572  ,  108.663    ,  127.837006 ,
        150.39299  ,  176.93001  ,  208.15204  ,  244.87505  ,
        288.08307  ,  337.5001   ,  375.0001   ,  412.5001   ,
        450.00012  ,  487.5001   ,  525.0001   ,  562.5001   ,
        600.0002   ,  637.5002   ,  675.0001   ,  700.0001   ,
        725.00006  ,  750.0001   ,  774.9998   ,  800.00024  ,
        820.0002   ,  835.0001   ,  850.0001   ,  865.       ,
        879.9999   ,  895.       ,  909.9998   ,  925.       ,
        940.0001   ,  954.9998   ,  969.99976  ,  985.       ,
       1000.       ], dtype=float32)

In [13]:
np.ediff1d(ds_mozart["pressure_edge"]["ilev"].values)

array([ 0.43418002,  0.5352411 ,  0.65621996,  0.8001399 ,  0.9702301 ,
        1.1699996 ,  1.4030414 ,  1.673099  ,  1.9839592 ,  2.3665    ,
        2.8136997 ,  3.3344994 ,  3.9388103 ,  4.6373997 ,  5.4421997 ,
        6.365402  ,  7.468498  ,  8.7439995 , 10.2155    , 11.908905  ,
       13.853409  , 16.29728   , 19.174004  , 22.555984  , 26.537018  ,
       31.22203   , 36.723007  , 43.208023  , 49.417023  , 37.5       ,
       37.5       , 37.50003   , 37.49997   , 37.50003   , 37.5       ,
       37.50006   , 37.5       , 37.49994   , 25.        , 24.999939  ,
       25.000061  , 24.999695  , 25.000427  , 19.999939  , 14.999939  ,
       15.        , 14.999878  , 14.999878  , 15.000122  , 14.999817  ,
       15.000183  , 15.000122  , 14.999695  , 14.999939  , 15.000244  ,
       15.        ], dtype=float32)

In [12]:
ds_geoschem["Met_PEDGE"].ilev.values

array([1.00000000e+00, 9.85000048e-01, 9.69999752e-01, 9.54999800e-01,
       9.40000110e-01, 9.25000010e-01, 9.09999810e-01, 8.95000010e-01,
       8.79999910e-01, 8.65000010e-01, 8.50000110e-01, 8.35000140e-01,
       8.20000180e-01, 8.00000220e-01, 7.74999820e-01, 7.50000110e-01,
       7.25000100e-01, 7.00000100e-01, 6.75000100e-01, 6.37500200e-01,
       6.00000200e-01, 5.62500100e-01, 5.25000100e-01, 4.87500100e-01,
       4.50000100e-01, 4.12500100e-01, 3.75000100e-01, 3.37500100e-01,
       2.88083060e-01, 2.44875040e-01, 2.08152025e-01, 1.76930008e-01,
       1.50393000e-01, 1.27837000e-01, 1.08663000e-01, 9.23657200e-02,
       7.85123100e-02, 5.63879100e-02, 4.01754100e-02, 2.83678100e-02,
       1.97916000e-02, 9.29294200e-03, 4.07657100e-03, 1.65079000e-03,
       6.16779100e-04, 2.11349000e-04, 6.60000100e-05, 1.00000000e-05])

In [15]:
np.ediff1d(ds_geoschem["Met_PEDGE"].ilev.values)

array([-1.49999517e-02, -1.50002963e-02, -1.49999520e-02, -1.49996900e-02,
       -1.50001000e-02, -1.50002000e-02, -1.49998000e-02, -1.50001000e-02,
       -1.49999000e-02, -1.49999000e-02, -1.49999700e-02, -1.49999600e-02,
       -1.99999600e-02, -2.50004000e-02, -2.49997100e-02, -2.50000100e-02,
       -2.50000000e-02, -2.50000000e-02, -3.74999000e-02, -3.75000000e-02,
       -3.75001000e-02, -3.75000000e-02, -3.75000000e-02, -3.75000000e-02,
       -3.75000000e-02, -3.75000000e-02, -3.75000000e-02, -4.94170400e-02,
       -4.32080200e-02, -3.67230150e-02, -3.12220168e-02, -2.65370082e-02,
       -2.25560000e-02, -1.91740000e-02, -1.62972800e-02, -1.38534100e-02,
       -2.21244000e-02, -1.62125000e-02, -1.18076000e-02, -8.57621000e-03,
       -1.04986580e-02, -5.21637100e-03, -2.42578100e-03, -1.03401090e-03,
       -4.05430100e-04, -1.45348990e-04, -5.60000100e-05])

In [14]:
# we need to compute actual pressure differences, not just the difference between ilev right?
# has mike already done this? does it need to be done separately at every location and time?
ds = xr.apply_ufunc(
    np.ediff1d,
    ds_mozart["pressure_edge"],
    input_core_dims=[["ilev"]],
    # output_core_dims=[["time", "lat", "lon"]],
    vectorize=True,
    dask="parallelized",
    # dask_gufunc_kwargs={
    #     "output_sizes": {"time": 22632, "ilev": 55, "lat": 180, "lon": 360}
    # }
)

ValueError: `dtype` inference failed in `apply_gufunc`.

Please specify the dtype explicitly using the `output_dtypes` kwarg.

Original error is below:
------------------------
ValueError('setting an array element with a sequence.')

Traceback:
---------
  File "/home/jj829/miniconda3/envs/nctoolkit_demo/lib/python3.9/site-packages/dask/array/core.py", line 455, in apply_infer_dtype
    o = func(*args, **kwargs)
  File "/home/jj829/miniconda3/envs/nctoolkit_demo/lib/python3.9/site-packages/numpy/lib/function_base.py", line 2304, in __call__
    return self._vectorize_call(func=func, args=vargs)
  File "/home/jj829/miniconda3/envs/nctoolkit_demo/lib/python3.9/site-packages/numpy/lib/function_base.py", line 2378, in _vectorize_call
    res = self._vectorize_call_with_signature(func, args)
  File "/home/jj829/miniconda3/envs/nctoolkit_demo/lib/python3.9/site-packages/numpy/lib/function_base.py", line 2438, in _vectorize_call_with_signature
    output[index] = result
